In [91]:
import torch
from torch.autograd import Variable
import numpy as np
import unittest
import torch.nn as nn
from scipy.special import erf

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [2]:
class Module(object):
    """
    Basically, you can think of a module as of a something (black box)
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`:

        output = module.forward(input)

    The module should be able to perform a backward pass: to differentiate the `forward` function.
    More, it should be able to differentiate it if is a part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule.

        gradInput = module.backward(input, gradOutput)
    """
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True

    def forward(self, input):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        return self.updateOutput(input)

    def backward(self,input, gradOutput):
        """
        Performs a backpropagation step through the module, with respect to the given input.

        This includes
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput


    def updateOutput(self, input):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.

        Make sure to both store the data in `output` field and return it.
        """

        # The easiest case:

        # self.output = input
        # return self.output

        pass

    def updateGradInput(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own input.
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.

        The shape of `gradInput` is always the same as the shape of `input`.

        Make sure to both store the gradients in `gradInput` field and return it.
        """

        # The easiest case:

        # self.gradInput = gradOutput
        # return self.gradInput

        pass

    def accGradParameters(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass

    def zeroGradParameters(self):
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass

    def getParameters(self):
        """
        Returns a list with its parameters.
        If the module does not have parameters return empty list.
        """
        return []

    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters.
        If the module does not have parameters return empty list.
        """
        return []

    def train(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = True

    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = False

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want
        to have readable description.
        """
        return "Module"

# Sequential container

**Define** a forward and backward pass procedures.

In [123]:
class Sequential(Module):
    """
         This class implements a container, which processes `input` data sequentially.

         `input` is processed by each module (layer) in self.modules consecutively.
         The resulting array is called `output`.
    """

    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []

    def add(self, module):
        """
        Adds a module to the container.
        """
        self.modules.append(module)

    def updateOutput(self, input):
        """
        Basic workflow of FORWARD PASS:

            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})


        Just write a little loop.
        """

        # Your code goes here. ################################################
        self.output = input
        for module in self.modules:
            self.output = module.forward(self.output)
        return self.output

    def backward(self, input, gradOutput):
        """
        Workflow of BACKWARD PASS:

            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)
            gradInput = module[0].backward(input, g_1)


        !!!

        To ech module you need to provide the input, module saw while forward pass,
        it is used while computing gradients.
        Make sure that the input for `i-th` layer the output of `module[i]` (just the same input as in forward pass)
        and NOT `input` to this Sequential module.

        !!!

        """
        # Your code goes here. ################################################
        modules_inputs = [input] + [module.output for module in self.modules[:-1]]

        self.gradInput = gradOutput
        for module, module_input in zip(self.modules[::-1], modules_inputs[::-1]):
            self.gradInput = module.backward(module_input, self.gradInput)

        return self.gradInput



    def zeroGradParameters(self):
        for module in self.modules:
            module.zeroGradParameters()

    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]

    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]

    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string

    def __getitem__(self,x):
        return self.modules.__getitem__(x)

    def train(self):
        """
        Propagates training parameter through all modules
        """
        self.training = True
        for module in self.modules:
            module.train()

    def evaluate(self):
        """
        Propagates training parameter through all modules
        """
        self.training = False
        for module in self.modules:
            module.evaluate()

# Layers

## 1 (0.2). Linear transform layer
Also known as dense layer, fully-connected layer, FC-layer, InnerProductLayer (in caffe), affine transform
- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

In [124]:
class Linear(Module):
    """
    A module which applies a linear transformation
    A common name is fully-connected layer, InnerProductLayer in caffe.

    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()

        # This is a nice initialization
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size = (n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size = n_out)

        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)
        
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.dot(input, self.W.T) + self.b   
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = np.dot(gradOutput, self.W)
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradW = np.dot(gradOutput.T, input) 
        self.gradb = np.sum(gradOutput, axis = 0)
        pass
    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)

    def getParameters(self):
        return [self.W, self.b]

    def getGradParameters(self):
        return [self.gradW, self.gradb]

    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

## 2. (0.2) SoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{softmax}(x)_i = \frac{\exp x_i} {\sum_j \exp x_j}$

Recall that $\text{softmax}(x) == \text{softmax}(x - \text{const})$. It makes possible to avoid computing exp() from large argument.

In [5]:
class SoftMax(Module):
    def __init__(self):
         super(SoftMax, self).__init__()

    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        exp_x = np.exp(self.output)
        exp_sum = np.sum(exp_x, axis=1, keepdims=True)
        self.output = exp_x / exp_sum
        return self.output

    def updateGradInput(self, input, gradOutput):
        z = self.output  
        grad = np.sum(gradOutput * z, axis=1, keepdims=True)  
        self.gradInput = z * (gradOutput - grad)
        return self.gradInput

    def __repr__(self):
        return "SoftMax"

## 3. (0.2) LogSoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{logsoftmax}(x)_i = \log\text{softmax}(x)_i = x_i - \log {\sum_j \exp x_j}$

The main goal of this layer is to be used in computation of log-likelihood loss.

In [125]:
class LogSoftMax(Module):
    def __init__(self):
         super(LogSoftMax, self).__init__()

    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        exp_x = np.exp(self.output)
        exp_sum = np.sum(exp_x, axis=1, keepdims=True)
        self.softmax = exp_x / exp_sum
        self.output = self.output - np.log(exp_sum)
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        grad = np.sum(gradOutput, axis=1, keepdims=True)
        self.gradInput = gradOutput - self.softmax * grad
        return self.gradInput

    def __repr__(self):
        return "LogSoftMax"

## 4. (0.3) Batch normalization
One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement the first part of the layer: features normalization. The second part (`ChannelwiseScaling` layer) is implemented below.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

The layer should work as follows. While training (`self.training == True`) it transforms input as $$y = \frac{x - \mu}  {\sqrt{\sigma + \epsilon}}$$
where $\mu$ and $\sigma$ - mean and variance of feature values in **batch** and $\epsilon$ is just a small number for numericall stability. Also during training, layer should maintain exponential moving average values for mean and variance:
```
    self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)
    self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)
```
During testing (`self.training == False`) the layer normalizes input using moving_mean and moving_variance.

Note that decomposition of batch normalization on normalization itself and channelwise scaling here is just a common **implementation** choice. In general "batch normalization" always assumes normalization + scaling.

In [7]:
class BatchNormalization(Module):
    EPS = 1e-3
    def __init__(self, alpha = 0.):
        super(BatchNormalization, self).__init__()
        self.alpha = alpha
        self.moving_mean = None
        self.moving_variance = None
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        # use self.EPS please
        if self.moving_mean is None:
            self.moving_mean = np.full(input.shape[1], 0)
            self.moving_variance = np.full(input.shape[1], 1)

        if self.training:
            batch_mean = np.mean(input, axis=0)  
            batch_variance = np.var(input, axis=0)  
            self.moving_mean = self.moving_mean * self.alpha + batch_mean * (1 - self.alpha)
            self.moving_variance = self.moving_variance * self.alpha + batch_variance * (1 - self.alpha)

            self.output = (input - batch_mean) / np.sqrt(batch_variance + self.EPS)
        
        else:
            mean = self.moving_mean
            var = self.moving_variance
            self.output = (input - mean) / np.sqrt(var + self.EPS)
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        batch_size = input.shape[0]
        
        self.gamma = np.full(input.shape[1], 1)

        batch_mean = np.mean(input, axis=0)
        batch_variance = np.var(input, axis=0)

        d_x_hat = gradOutput * self.gamma
        d_var = np.sum(d_x_hat * (input - batch_mean) * -0.5 * (1.0 / np.sqrt(batch_variance + self.EPS))**3, axis=0)
        d_mean = np.sum(d_x_hat * (-1.0 / np.sqrt(batch_variance + self.EPS)), axis=0) + d_var * np.mean(-2.0 * (input - batch_mean), axis=0)
        self.gradInput = d_x_hat * (1.0 / np.sqrt(batch_variance + self.EPS)) + (d_var * 2 * (input - batch_mean) / batch_size) + (d_mean / batch_size)
        
        return self.gradInput
        
    def train(self):
        self.training = True

    def evaluate(self):
        self.training = False
    def __repr__(self):
        return "BatchNormalization"

In [8]:
class ChannelwiseScaling(Module):
    """
       Implements linear transform of input y = \gamma * x + \beta
       where \gamma, \beta - learnable vectors of length x.shape[-1]
    """
    def __init__(self, n_out):
        super(ChannelwiseScaling, self).__init__()

        stdv = 1./np.sqrt(n_out)
        self.gamma = np.random.uniform(-stdv, stdv, size=n_out)
        self.beta = np.random.uniform(-stdv, stdv, size=n_out)

        self.gradGamma = np.zeros_like(self.gamma)
        self.gradBeta = np.zeros_like(self.beta)

    def updateOutput(self, input):
        self.output = input * self.gamma + self.beta
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * self.gamma
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        self.gradBeta = np.sum(gradOutput, axis=0)
        self.gradGamma = np.sum(gradOutput*input, axis=0)

    def zeroGradParameters(self):
        self.gradGamma.fill(0)
        self.gradBeta.fill(0)

    def getParameters(self):
        return [self.gamma, self.beta]

    def getGradParameters(self):
        return [self.gradGamma, self.gradBeta]

    def __repr__(self):
        return "ChannelwiseScaling"

<>:2: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Valee\AppData\Local\Temp\ipykernel_11452\3092293347.py:2: SyntaxWarning: invalid escape sequence '\g'
  """


Practical notes. If BatchNormalization is placed after a linear transformation layer (including dense layer, convolutions, channelwise scaling) that implements function like `y = weight * x + bias`, than bias adding become useless and could be omitted since its effect will be discarded while batch mean subtraction. If BatchNormalization (followed by `ChannelwiseScaling`) is placed before a layer that propagates scale (including ReLU, LeakyReLU) followed by any linear transformation layer than parameter `gamma` in `ChannelwiseScaling` could be freezed since it could be absorbed into the linear transformation layer.

## 5. (0.3) Dropout
Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. Here $p$ is probability of an element to be zeroed.

This has proven to be an effective technique for regularization and preventing the co-adaptation of neurons.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch), zero out elements and multiply elements by $1 / (1 - p)$. The latter is needed for keeping mean values of features close to mean values which will be in test mode. When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [9]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()

        self.p = p
        self.mask = None

    def updateOutput(self, input):
        # Your code goes here. ################################################
        if self.training:
            d = np.random.uniform(0, 1, input.shape) 
            self.mask = d > self.p
            self.output = (1/(1-self.p)) * self.mask * input
        else:
            self.output = input
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        if self.training:
            self.gradInput = (1/(1-self.p)) * gradOutput * self.mask
        else:
            self.gradInput = gradOutput
        return self.gradInput
    
    def train(self):
        self.training = True

    def evaluate(self):
        self.training = False
        
    def __repr__(self):
        return "Dropout"

## 6. (2.0) Conv2d
Implement [**Conv2d**](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html). Use only this list of parameters: (in_channels, out_channels, kernel_size, stride, padding, bias, padding_mode) and fix dilation=1 and groups=1.

In [101]:
class Conv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=0, bias=True, padding_mode='zeros'):
        super(Conv2d, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size
        self.stride = stride
        self.padding = padding
        self.bias = bias
        self.padding_mode = padding_mode

    def updateOutput(self, input):
    
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        return self.gradInput

    def __repr__(self):
        return "Conv2d"

## 7. (0.5) 
Implement [**MaxPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) and [**AvgPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html). Use only parameters like kernel_size, stride, padding (negative infinity for maxpool and zero for avgpool) and other parameters fixed as in framework.

In [115]:
class MaxPool2d(Module):
    def __init__(self, kernel_size, stride, padding):
        super(MaxPool2d, self).__init__()

        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

    def updateOutput(self, input):
        # Your code goes here. ################################################
        if isinstance(self.kernel_size, int):
            self.kernel_size = (self.kernel_size, self.kernel_size)
            
        if isinstance(self.stride, int):
            self.stride = (self.stride, self.stride)
            
        if len(input.shape) == 3:  
            input = np.expand_dims(input, axis=1)
        input = input.copy()
        
        self.output = np.zeros((input.shape[0], input.shape[1], int(input.shape[2] / self.kernel_size[0]), int(input.shape[3] / self.kernel_size[1])))
        self.max_indices = np.zeros((input.shape[0], input.shape[1], int(input.shape[2] / self.kernel_size[0]), int(input.shape[3] / self.kernel_size[1]), 2), dtype=int)
        
        for i in range(input.shape[0]):
            for j in range(input.shape[1]): 
                result = []
                self.i = 0
                out_i = 0
                while self.i < input[i][j].shape[0] - self.kernel_size[0] + 1:
                    self.j = 0
                    out_j = 0
                    while self.j < input[i][j].shape[1] - self.kernel_size[1] + 1:
                        result.append(np.max(input[i][j][self.i:self.i + self.kernel_size[0], self.j:self.j + self.kernel_size[1]]))
                        max_pos = np.unravel_index(np.argmax(input[i][j][self.i:self.i + self.kernel_size[0], self.j:self.j + self.kernel_size[1]]), (self.kernel_size[0], self.kernel_size[1]))
                        self.max_indices[i][j][out_i, out_j] = (self.i + max_pos[0], self.j + max_pos[1])
                        
                        input[i][j][self.i:self.i + self.kernel_size[0], self.j:self.j + self.kernel_size[1]] = (
                            input[i][j][self.i:self.i + self.kernel_size[0], self.j:self.j + self.kernel_size[1]] * [input[i][j][self.i:self.i + self.kernel_size[0], self.j:self.j + self.kernel_size[1]] == 
                             np.max(input[i][j][self.i:self.i + self.kernel_size[0], self.j:self.j + self.kernel_size[1]])])
    
                        self.j += self.stride[1] 
                        out_j += 1  
                    self.i += self.stride[0] 
                    out_i += 1  
                self.output[i][j] = np.array(result).reshape(int(input[i][j].shape[0] / self.kernel_size[0]), int(input[i][j].shape[1] / self.kernel_size[1]))

        self.input = input
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.zeros_like(input)

        batch_size, channels, height, width = input.shape
        out_height = (height - self.kernel_size[0]) // self.stride[0] + 1
        out_width = (width - self.kernel_size[1]) // self.stride[1] + 1
 
        for i in range(out_height):
            for j in range(out_width):
                h_start = i * self.stride[0]
                h_end = h_start + self.kernel_size[0]
                w_start = j * self.stride[1]
                w_end = w_start + self.kernel_size[1]

                max_h = self.max_indices[:, :, i, j, 0]  
                max_w = self.max_indices[:, :, i, j, 1]  

                self.gradInput[np.arange(batch_size)[:, None], np.arange(channels)[None, :], max_h, max_w] += gradOutput[:, :, i, j]

        if self.padding > 0:
            self.gradInput = self.gradInput[:, :, self.padding:-self.padding, self.padding:-self.padding]

        return self.gradInput

    def __repr__(self):
        return "MaxPool2d"

In [116]:
class AvgPool2d(Module):
    def __init__(self, kernel_size, stride, padding):
        super(AvgPool2d, self).__init__()

        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

    def updateOutput(self, input):
        # Your code goes here. ################################################
        if isinstance(self.kernel_size, int):
            self.kernel_size = (self.kernel_size, self.kernel_size)
        if isinstance(self.stride, int):
            self.stride = (self.stride, self.stride)
        if isinstance(self.padding, int):
            self.padding = (self.padding, self.padding)

        padded_input = np.pad(input, 
                            ((0, 0), (0, 0),
                            (self.padding[0], self.padding[0]),
                            (self.padding[1], self.padding[1])),
                            mode='constant')
        
        batch_size, channels = input.shape[0], input.shape[1]
        out_h = (input.shape[2] + 2*self.padding[0] - self.kernel_size[0]) // self.stride[0] + 1
        out_w = (input.shape[3] + 2*self.padding[1] - self.kernel_size[1]) // self.stride[1] + 1
        
        self.output = np.zeros((batch_size, channels, out_h, out_w))
        self.window_areas = np.zeros((batch_size, channels, out_h, out_w))
        
        for b in range(batch_size):
            for c in range(channels):
                out_i = 0
                i = 0
                while i <= padded_input.shape[2] - self.kernel_size[0]:
                    out_j = 0
                    j = 0
                    while j <= padded_input.shape[3] - self.kernel_size[1]:
                        window = padded_input[b, c, i:i+self.kernel_size[0], j:j+self.kernel_size[1]]
                        self.output[b, c, out_i, out_j] = np.mean(window)
                        self.window_areas[b, c, out_i, out_j] = 1.0 / (self.kernel_size[0] * self.kernel_size[1])
                        j += self.stride[1]
                        out_j += 1
                    i += self.stride[0]
                    out_i += 1
        
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        batch_size, channels = input.shape[0], input.shape[1]
        
        gradInput = np.zeros_like(input)
        padded_grad = np.pad(gradInput,
                           ((0, 0), (0, 0),
                           (self.padding[0], self.padding[0]),
                           (self.padding[1], self.padding[1])),
                           mode='constant')
        
        for b in range(batch_size):
            for c in range(channels):
                out_i = 0
                i = 0
                while i <= input.shape[2] + 2*self.padding[0] - self.kernel_size[0]:
                    out_j = 0
                    j = 0
                    while j <= input.shape[3] + 2*self.padding[1] - self.kernel_size[1]:
                        grad = gradOutput[b, c, out_i, out_j] * self.window_areas[b, c, out_i, out_j]
                        padded_grad[b, c, i:i+self.kernel_size[0], j:j+self.kernel_size[1]] += grad
                        j += self.stride[1]
                        out_j += 1
                    i += self.stride[0]
                    out_i += 1
        if self.padding[0] > 0 or self.padding[1] > 0:
            self.gradInput = padded_grad[:, :, self.padding[0]:-self.padding[0], self.padding[1]:-self.padding[1]]
        else:
            self.gradInput = padded_grad
        
        return self.gradInput

    def __repr__(self):
        return "AvgPool2d"

## 8. (0.3) 
Implement **GlobalMaxPool2d** and **GlobalAvgPool2d**. They do not have testing and parameters are up to you but they must aggregate information within channels. Write test functions for these layers on your own.

In [85]:
class GlobalMaxPool2d:
    def __init__(self):
        self.input = None
        self.output = None
        self.max_indices = None
    
    def updateOutput(self, input):
        self.input = input
        batch_size, channels = input.shape[0], input.shape[1]
        self.output = np.zeros((batch_size, channels, 1, 1))
        self.max_indices = np.zeros((batch_size, channels, 2), dtype=int)
        
        for i in range(batch_size):
            for j in range(channels):
                max_val = -np.inf
                max_h, max_w = 0, 0
                for h in range(input.shape[2]):
                    for w in range(input.shape[3]):
                        if input[i, j, h, w] > max_val:
                            max_val = input[i, j, h, w]
                            max_h, max_w = h, w
                self.output[i, j, 0, 0] = max_val
                self.max_indices[i, j] = [max_h, max_w]
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        gradInput = np.zeros_like(input)
        batch_size, channels = input.shape[0], input.shape[1]
        
        for i in range(batch_size):
            for j in range(channels):
                h, w = self.max_indices[i, j]
                gradInput[i, j, h, w] = gradOutput[i, j, 0, 0]
        
        return gradInput

    def __repr__(self):
        return "GlobalMaxPool2d()"

In [86]:
class GlobalAvgPool2d:
    def __init__(self):
        self.input = None
        self.output = None
        self.input_shape = None
    
    def updateOutput(self, input):
        self.input = input
        self.input_shape = input.shape
        self.output = np.mean(input, axis=(2, 3), keepdims=True)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        gradInput = np.zeros_like(input)
        batch_size, channels, h, w = input.shape
        nf = 1.0 / (h * w)
        for i in range(batch_size):
            for j in range(channels):
                gradInput[i, j, :, :] = gradOutput[i, j, 0, 0] * nf
        return gradInput

    def __repr__(self):
        return "GlobalAvgPool2d()"

## 9. (0.2) 
Implement [**Flatten**](https://pytorch.org/docs/stable/generated/torch.flatten.html)

In [117]:
class Flatten(Module):
    def __init__(self, start_dim=0, end_dim=-1):
        super(Flatten, self).__init__()

        self.start_dim = start_dim
        self.end_dim = end_dim

    def updateOutput(self, input):
        # Your code goes here. ################################################
        if self.end_dim == -1:
            self.end_dim = len(input.shape)
        self.output = input.reshape(*input.shape[:self.start_dim], np.prod(input.shape[self.start_dim : self.end_dim + 1]),*input.shape[self.end_dim + 1:])
        return self.output


    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = gradOutput.reshape(input.shape)
        return self.gradInput

    def __repr__(self):
        return "Flatten"
    

# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**):

In [140]:
class ReLU(Module):
    def __init__(self, slope=0.01):
        super(ReLU, self).__init__()
        self.slope = slope

    def updateOutput(self, input):
        self.output = np.maximum(input, self.slope * input)
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * (input > 0)
        return self.gradInput

## 10. (0.1) Leaky ReLU
Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope.

In [130]:
class LeakyReLU(Module):
    def __init__(self, slope = 0.03):
        super(LeakyReLU, self).__init__()
        self.slope = slope

    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.maximum(input, self.slope * input)
        return self.output
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        gard = np.where(input > 0, 1, self.slope)
        self.gradInput = np.multiply(gradOutput, gard)
        return self.gradInput

    def __repr__(self):
        return "LeakyReLU"

## 11. (0.1) ELU
Implement [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activations.

In [132]:
class ELU(Module):
    def __init__(self, alpha = 1.0):
        super(ELU, self).__init__()

        self.alpha = alpha

    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.where(input > 0, input, self.alpha * (np.exp(input) - 1))
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        gard = np.where(input >= 0, 1, self.output + self.alpha)
        self.gradInput = gradOutput * gard
        return self.gradInput

    def __repr__(self):
        return "ELU"

## 12. (0.1) SoftPlus
Implement [**SoftPlus**](https://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29) activations. Look, how they look a lot like ReLU.

In [17]:
class SoftPlus(Module):
    def __init__(self):
        super(SoftPlus, self).__init__()

    def updateOutput(self, input):
        # Your code goes here. ################################################
        exp_x = np.exp(input)
        self.output = np.log(1 + exp_x)
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        grad = 1 / (1 + np.exp(-input))
        self.gradInput = gradOutput * grad
        return self.gradInput

    def __repr__(self):
        return "SoftPlus"

## 13. (0.2) Gelu
Implement [**Gelu**](https://pytorch.org/docs/stable/generated/torch.nn.GELU.html) activations.

In [98]:
class Gelu(Module):
    def __init__(self):
        super(Gelu, self).__init__()

    def updateOutput(self, input):
        # Your code goes here. ################################################
        factor = np.sqrt(2.0 / np.pi)
        tanh_x = (np.exp(factor * (input + 0.044715 * input**3)) - np.exp(-factor * (input + 0.044715 * input**3))) / \
                 (np.exp(factor * (input + 0.044715 * input**3)) + np.exp(-factor * (input + 0.044715 * input**3)))
        self.output = 0.5 * input * (1 + tanh_x)
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        coeff = np.sqrt(2 / np.pi)
        inner = coeff * (input + 0.044715 * np.power(input, 3))
        tanh = np.tanh(inner)
        d_L = 0.5 * (1 + tanh) + 0.5 * input * (1 - tanh ** 2) * coeff * (1 + 3 * 0.044715 * np.power(input, 2))

        self.gradInput = gradOutput * d_L
        return self.gradInput

    def __repr__(self):
        return "Gelu"

# Criterions

Criterions are used to score the models answers.

In [19]:
class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None

    def forward(self, input, target):
        """
            Given an input and a target, compute the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        return self.updateGradInput(input, target)

    def updateOutput(self, input, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        return self.gradInput

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want
        to have readable description.
        """
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.
- input:   **`batch_size x n_feats`**
- target: **`batch_size x n_feats`**
- output: **scalar**

In [63]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()

    def updateOutput(self, input, target):
        self.output = np.sum(np.power(input - target,2)) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        self.gradInput  = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

## 14. (0.2) Negative LogLikelihood criterion (numerically unstable)
You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula,
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. Also there is a small hack with adding small number to probabilities to avoid computing log(0).
- input:   **`batch_size x n_feats`** - probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**



In [48]:
class ClassNLLCriterionUnstable(Criterion):
    EPS = 1e-15
    def __init__(self):
        # a = super(ClassNLLCriterionUnstable, self)
        super(ClassNLLCriterionUnstable, self).__init__()

    def updateOutput(self, input, target):

        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        # Your code goes here. ################################################

        self.output = -np.sum(target*np.log(input_clamp)) / input.shape[0] 
        return self.output

    def updateGradInput(self, input, target):
        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)

        # Your code goes here. ###############################################
        self.gradInput = -target / input_clamp / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterionUnstable"

## 15. (0.3) Negative LogLikelihood criterion (numerically stable)
- input:   **`batch_size x n_feats`** - log probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**

Task is similar to the previous one, but now the criterion input is the output of log-softmax layer. This decomposition allows us to avoid problems with computation of forward and backward of log().

In [51]:
class ClassNLLCriterion(Criterion):
    EPS = 1e-15
    def __init__(self):
        a = super(ClassNLLCriterion, self)
        super(ClassNLLCriterion, self).__init__()

    def updateOutput(self, input, target):
        # Your code goes here. ################################################
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)

        self.output = -np.sum(target * input) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        # Your code goes here. ################################################
        self.gradInput = -target / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterion"

1-я часть задания: реализация слоев, лосей и функций активации - 5 баллов. \\
2-я часть задания: реализация моделей на своих классах. Что должно быть:
  1. Выберите оптимизатор и реализуйте его, чтоб он работал с вами классами. - 1 балл.
  2. Модель для задачи мультирегрессии на выбраных вами данных. Использовать FCNN, dropout, batchnorm, MSE. Пробуйте различные фукнции активации. Для первой модели попробуйте большую, среднюю и маленькую модель. - 1 балл.
  3. Модель для задачи мультиклассификации на MNIST. Использовать свёртки, макспулы, флэттэны, софтмаксы - 1 балла.
  4. Автоэнкодер для выбранных вами данных. Должен быть на свёртках и полносвязных слоях, дропаутах, батчнормах и тд. - 2 балла. \\

Дополнительно в оценке каждой модели будет учитываться:
1. Наличие правильно выбранной метрики и лосс функции.
2. Отрисовка графиков лосей и метрик на трейне-валидации. Проверка качества модели на тесте.
3. Наличие шедулера для lr.
4. Наличие вормапа.
5. Наличие механизма ранней остановки и сохранение лучшей модели.
6. Свитч лося (метрики) и оптимайзера.